In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE

In [ ]:
loans = pd.read_csv('data/lending-club-data.csv')
loans.iloc[0]

In [ ]:
loans.bad_loans.value_counts()

In [ ]:
loans = loans[~loans.payment_inc_ratio.isnull()]

In [ ]:
model_variables = ['grade', 'home_ownership','emp_length_num', 'sub_grade','short_emp',
            'dti', 'term', 'purpose', 'int_rate', 'last_delinq_none', 'last_major_derog_none',
            'revol_util', 'total_rec_late_fee', 'payment_inc_ratio', 'bad_loans']

loans_data_relevent = loans[model_variables]

In [ ]:
loans_relevant_enconded = pd.get_dummies(loans_data_relevent)

In [ ]:
loans_relevant_enconded.bad_loans.value_counts(normalize=True)

### Creating the Training and Test Sets

In [ ]:
training_features, test_features, \
training_target, test_target, = train_test_split(loans_relevant_enconded.drop(['bad_loans'], axis=1),
                                               loans_relevant_enconded['bad_loans'],
                                               test_size = .1,
                                               random_state=12)

### Wrong way 

In [ ]:
sm = SMOTE(random_state=12)
x_res, y_res = sm.fit_sample(training_features, training_target)
print (training_target.value_counts(), np.bincount(y_res))

In [ ]:
x_train_res, x_val_res, y_train_res, y_val_res = train_test_split(x_res,
                                                    y_res,
                                                    test_size = .1,
                                                    random_state=12)

In [ ]:
clf_rf = RandomForestClassifier(n_estimators=25, random_state=12)
clf_rf.fit(x_train_res, y_train_res)
clf_rf.score(x_val_res, y_val_res)

In [ ]:
print(classification_report(y_val_res, clf_rf.predict(x_val_res)))

In [ ]:
print (clf_rf.score(test_features, test_target))
print (recall_score(test_target, clf_rf.predict(test_features)))

#### Reason for bad output since its Bleeds Information 

### Right Approach

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(training_features, training_target,
                                                  test_size = .1,
                                                  random_state=12)

In [ ]:
sm = SMOTE(random_state=12)
x_train_res, y_train_res = sm.fit_sample(x_train, y_train)

In [ ]:
clf_rf = RandomForestClassifier(n_estimators=25, random_state=12)
clf_rf.fit(x_train_res, y_train_res)

In [ ]:
print ('Validation Results')
print (clf_rf.score(x_val, y_val))
print (recall_score(y_val, clf_rf.predict(x_val)))
print ('\nTest Results')
print (clf_rf.score(test_features, test_target))
print (recall_score(test_target, clf_rf.predict(test_features)))